# Scraper

In [1]:
!pip install selenium
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
url = 'https://www.chrono24.ca/rolex/index-1.htm?pageSize=120&showpage=1'

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

# IP check
url_check_ip = 'https://api.ipify.org/?format=json'
driver.get(url_check_ip)
page_source = driver.page_source
soup = BeautifulSoup(page_source, "html.parser")
ip_address = soup.find('body').text
print(ip_address)

# Try first page
driver.get(url)
html = driver.page_source
driver.quit()

{"ip":"34.27.129.213"}


## Scrape URLs

In [51]:
soup = BeautifulSoup(html, 'html.parser')
# soup

In [53]:
urls = []
result_container = soup.find_all('div', class_='js-article-item-container article-item-container wt-search-result article-image-carousel')
for container in result_container:
    a_tag = container.find('a')
    if a_tag and a_tag.has_attr('href'):
        urls.append(a_tag['href'])
print(len(urls))
urls[0:4]

122


['/rolex/rolex-datejust-41-18k-rose-gold--steel-full-set-2021-126331--id31679404.htm',
 '/rolex/rolex-submariner-date-126610ln-black-oyster-steel-41-mm--id32479762.htm',
 '/rolex/rolex-oyster-perpetual-41---124300-green-new-2023--id29944257.htm',
 '/rolex/rolex-submariner-ref-5512--id32739931.htm']

In [46]:
pages = soup.find('ul', class_="pagination list-unstyled d-flex pull-xs-none pull-sm-right")
page_tags = pages.find_all('a',
    'onclick'=="tce('search.manufacturerlist', 'search-page-link-bottom');",
    href=True)
last_page_num = int(page_tags[-1].text.strip())
last_page_num

835

In [54]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

options = webdriver.ChromeOptions()

# Config to work on Google Colab
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Pretend to be a non-headless browser
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
options.add_argument("--lang=en-US")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1280,720")

driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10) # Wait for elements to load

urls = []

for n in range(1, 10 + 1):
    target_url = f'https://www.chrono24.ca/rolex/index-{n}.htm?pageSize=120&showpage=1'
    driver.get(target_url)
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    result_container = soup.find_all('div', class_='js-article-item-container article-item-container wt-search-result article-image-carousel') # article-item-container wt-search-result article-image-carousel
    url_list_length = len(urls)

    for container in result_container:
        a_tag = container.find('a')
        if a_tag and a_tag.has_attr('href'):
            urls.append('https://www.chrono24.ca' + a_tag['href'])

    if len(urls) == url_list_length:  # if nothing is returned then your IP is blocked
        print(f'Connection lost at page number {n}. {len(urls)} URLs collected.')
        pd.DataFrame(urls, columns=['url']).to_csv(f'/content/drive/MyDrive/Colab Notebooks/chrono/data/url_df_until_page{n}.csv')
        break

    print(f'Page {n} completed - URL length: {len(urls)}')

driver.quit()

# Save the URLs to a CSV file
pd.DataFrame(urls, columns=['url']).to_csv(f'/content/drive/MyDrive/Colab Notebooks/chrono/data/url_df_complete.csv')
print(f'{len(urls)} URLs retrieved')


Page 1 completed - URL length: 123
Page 2 completed - URL length: 243
Page 3 completed - URL length: 363
Page 4 completed - URL length: 483
Page 5 completed - URL length: 603
Page 6 completed - URL length: 723
Page 7 completed - URL length: 843
Page 8 completed - URL length: 963
Page 9 completed - URL length: 1083
Page 10 completed - URL length: 1203
1203 URLs retrieved


In [55]:
# urls = ['https://www.chrono24.ca/' + url for url in urls]
urls[:5]

['https://www.chrono24.ca//rolex/rolex-oyster-perpetual-31-2023-new-release-celebration-bubble-tiffany-dial---complete-set-31mm--id29817524.htm',
 'https://www.chrono24.ca//rolex/rolex-datejust-126334-41mm-blue-index-dial-jubilee-bracelet-unworn-2023--id26225223.htm',
 'https://www.chrono24.ca//rolex/rolex-2023-unworn-126718-gmt-master-ii-18k-yellow-gold-40mm-black-dial--jubilee-bracelet--id31408320.htm',
 'https://www.chrono24.ca//rolex/rolex-submariner-ref-5512--id32739931.htm',
 'https://www.chrono24.ca//rolex/rolex-sky-dweller-336934-blue-new-2024-jubilee-steel-42-mm--id31354865.htm']

# Scrape data

In [6]:
complete_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/chrono/data/complete_url_df.csv')

In [7]:
complete_df.shape

(75955, 1)

In [23]:
options = webdriver.ChromeOptions()

# Config to work on Google Colab
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# Pretend to be a non-headless browser
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")
options.add_argument("--lang=en-US")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--window-size=1280,720")

driver = webdriver.Chrome(options=options)

In [24]:
driver.get('https://www.chrono24.ca/rolex/rolex-factory-diamond-dial--id33076260.htm')
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
len(str(soup))

475853

In [25]:
start_num = 722
result = {}
try:
  # for i in range(complete_df.shape[0]):
  for i in range(start_num, 10000):
    data_url = complete_df.iloc[i, 0]
    driver.get(data_url)
    # time.sleep(np.random.randint(1,10))
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
      title = soup.find('h1', class_="h3 m-y-0").contents[0].strip()
    except:
      title = ''
    try:
      subtitle = soup.find('p', class_="text-md text-sm-lg").contents[0].strip()
    except:
      subtitle = ''
    try:
      rating = soup.find('span', class_='rating').contents[2].strip()
    except:
      rating = 0
    try:
      reviews = soup.find('button', class_="js-link-merchant-reviews link").contents[0].strip()
    except:
      reviews = 0
    try:
      description = soup.find('span', id='watchNotes').get_text(strip=True)
    except:
      description = ''

    content_table = soup.find('table')

    extracted_info = []
    # Loop through each tbody element to handle sections separately
    for tbody in content_table.find_all('tbody'):
        # Check if the section is 'Functions' by looking for a h3 tag
        section_header = tbody.find('h3')
        if section_header and 'Functions' in section_header.text:
            # Directly add the 'Functions' value
            functions = tbody.find('tr').find_next_sibling('tr').text.strip()
            extracted_info.append(('Functions', functions))
        else:
            # Extract other table data
            for tr in tbody.find_all('tr'):
                th = tr.find('strong')
                if th:
                    key = th.text.strip()
                    # Some values might be in 'a' tags or directly in 'td'
                    value = tr.find('td').find_next_sibling('td').text.strip()
                    extracted_info.append((key, value))

    # add more info outside the table
    extracted_info.append(('title', title))
    extracted_info.append(('subtitle', subtitle))
    extracted_info.append(('rating', rating))
    extracted_info.append(('reviews', reviews))
    extracted_info.append(('description', description))

    result[i+1] = extracted_info
    print(f'URL {i+1} completed')

    flattened_data = {key: dict(value) for key, value in result.items()}
    result_df = pd.DataFrame.from_dict(flattened_data, orient='index')
    result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/chrono/data/result_df_completed.csv')

except:
  flattened_data = {key: dict(value) for key, value in result.items()}
  result_df = pd.DataFrame.from_dict(flattened_data, orient='index')
  result_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/chrono/data/result_df_until_{result_df.shape[0] + start_num}.csv')

URL 723 completed
URL 724 completed
URL 725 completed
URL 726 completed
URL 727 completed
URL 728 completed
URL 729 completed
URL 730 completed
URL 731 completed
URL 732 completed
URL 733 completed
URL 734 completed
URL 735 completed
URL 736 completed
URL 737 completed
URL 738 completed
URL 739 completed
URL 740 completed
URL 741 completed
URL 742 completed
URL 743 completed
URL 744 completed
URL 745 completed
URL 746 completed
URL 747 completed
URL 748 completed
URL 749 completed
URL 750 completed
URL 751 completed
URL 752 completed
URL 753 completed
URL 754 completed
URL 755 completed
URL 756 completed
URL 757 completed
URL 758 completed
URL 759 completed
URL 760 completed
URL 761 completed
URL 762 completed
URL 763 completed
URL 764 completed
URL 765 completed
URL 766 completed
URL 767 completed
URL 768 completed
URL 769 completed
URL 770 completed
URL 771 completed
URL 772 completed
URL 773 completed
URL 774 completed
URL 775 completed
URL 776 completed
URL 777 completed
URL 778 co

In [26]:
flattened_data = {key: dict(value) for key, value in result.items()}
result_df = pd.DataFrame.from_dict(flattened_data, orient='index')

In [27]:
result_df.to_csv(f'/content/drive/MyDrive/Colab Notebooks/chrono/data/result_df_until_{result_df.shape[0] + start_num}.csv')

In [28]:
result_df.tail()

,Listing code,Brand,Model,Reference number,Movement,Case material,Bracelet material,Year of production,Condition,Scope of delivery,...,Functions,Frequency,Bracelet thickness,Lug width,Thickness,Bracelet length,Buckle width,Alternative reference numbers,"Model, reference number",Dealer product code
1049,JANCG2,Rolex,Oyster Perpetual 41,124300,Automatic,Steel,Steel,2023,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1050,JKX381,Rolex,Datejust 41,126300,Automatic,Steel,Steel,2024,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1051,GMN140,Rolex,Datejust 41,126300,Automatic,Steel,Steel,2022,"New\n(Brand new, without any signs of wear)","Original box, original papers",...,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
980,NaN,Rolex,Day-Date 36,118238,Automatic,Yellow gold,Yellow gold,"2000, 2001, 2005, 2006, 2007",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"118238-0103, 118238-0116, 118238A, 118238 (G)",NaN,NaN
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Price (approx.),NaN


dev

In [13]:
data_url_test = complete_df.iloc[0, 0]

driver.get(data_url_test)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [103]:
title = soup.find('h1', class_="h3 m-y-0").contents[0].strip()
subtitle = soup.find('p', class_="text-md text-sm-lg").contents[0].strip()
rating = soup.find('span', class_='rating').contents[2].strip()
reviews = soup.find('button', class_="js-link-merchant-reviews link").contents[0].strip()
description = soup.find('span', id='watchNotes').get_text(strip=True)
content_table = soup.find('table')

In [97]:
# Initialize a list to store the extracted information
extracted_info = []

# Loop through each tbody element to handle sections separately
for tbody in content_table.find_all('tbody'):
    # Check if the section is 'Functions' by looking for a h3 tag
    section_header = tbody.find('h3')
    if section_header and 'Functions' in section_header.text:
        # Directly add the 'Functions' value
        functions = tbody.find('tr').find_next_sibling('tr').text.strip()
        extracted_info.append(('Functions', functions))
    else:
        # Extract other table data
        for tr in tbody.find_all('tr'):
            th = tr.find('strong')
            if th:
                key = th.text.strip()
                # Some values might be in 'a' tags or directly in 'td'
                value = tr.find('td').find_next_sibling('td').text.strip()
                extracted_info.append((key, value))

# add more info outside the table
extracted_info.append(('title', title))
extracted_info.append(('subtitle', subtitle))
extracted_info.append(('rating', rating))
extracted_info.append(('reviews', reviews))
extracted_info.append(('description', description))

Attribute,Brand,Model,Reference number,Movement,Case material,Bracelet material,Year of production,Condition,Scope of delivery,Gender,...,Water resistance,Bezel material,Crystal,Dial,Dial numerals,Bracelet material,Bracelet color,Clasp,Clasp material,Functions
Listing code,,,,,,,,,,,,,,,,,,,,,
JKH705,Rolex,GMT-Master II,126711CHNR,Automatic,Gold/Steel,Gold/Steel,2023,"Unworn\n(Mint condition, without signs of wear)","Original box, original papers",Men's watch/Unisex,...,10 ATM,Ceramic,Sapphire crystal,Black,No numerals,Gold/Steel,Gold/Steel,Fold clasp,Gold/Steel,"Date, GMT"


In [111]:
extracted_info

[('Listing code', 'JKH705'),
 ('Brand', 'Rolex'),
 ('Model', 'GMT-Master II'),
 ('Reference number', '126711CHNR'),
 ('Movement', 'Automatic'),
 ('Case material', 'Gold/Steel'),
 ('Bracelet material', 'Gold/Steel'),
 ('Year of production', '2023'),
 ('Condition', 'Unworn\n(Mint condition, without signs of wear)'),
 ('Scope of delivery', 'Original box, original papers'),
 ('Gender', "Men's watch/Unisex"),
 ('Location', 'United States of America, New York, New York'),
 ('Price', '$19,495 (= C$26,929) [Negotiable]'),
 ('Availability', 'Item is in stock'),
 ('Movement', 'Automatic'),
 ('Caliber/movement', '3285'),
 ('Base caliber', 'Rolex 3285'),
 ('Power reserve', '70 h'),
 ('Number of jewels', '31'),
 ('Case material', 'Gold/Steel'),
 ('Case diameter', '40 mm\n\nTry it on'),
 ('Water resistance', '10 ATM'),
 ('Bezel material', 'Ceramic'),
 ('Crystal', 'Sapphire crystal'),
 ('Dial', 'Black'),
 ('Dial numerals', 'No numerals'),
 ('Bracelet material', 'Gold/Steel'),
 ('Bracelet color', 'Gol

In [112]:
# Convert the list of tuples into a DataFrame
df = pd.DataFrame(extracted_info, columns=['Attribute', 'Value']).set_index('Attribute').T.set_index('Listing code')
df

Attribute,Brand,Model,Reference number,Movement,Case material,Bracelet material,Year of production,Condition,Scope of delivery,Gender,...,Bracelet material,Bracelet color,Clasp,Clasp material,Functions,title,subtitle,rating,reviews,description
Listing code,,,,,,,,,,,,,,,,,,,,,
JKH705,Rolex,GMT-Master II,126711CHNR,Automatic,Gold/Steel,Gold/Steel,2023,"Unworn\n(Mint condition, without signs of wear)","Original box, original papers",Men's watch/Unisex,...,Gold/Steel,Gold/Steel,Fold clasp,Gold/Steel,"Date, GMT",Rolex GMT-Master II,Unworn GMT-Master II 40mm Rootbeer Two Tone Ro...,5.0,4,Explore the world of unworn Rolex watches with...
